# Import libraries

In [1]:
import numpy as np
import random

# Importing data

Import the banknote data as 'data'. A single datapoint corresponds to one row of the txt file.
each feature are separated by commas, the last integer value for each row is the label.

In [2]:
data = np.loadtxt("data_banknote_authentication.txt", delimiter=",")

### Example

Here i get a feel for what the data i will be working with, looks like 

In [3]:
print(data.shape)
print(data)

(1372, 5)
[[  3.6216    8.6661   -2.8073   -0.44699   0.     ]
 [  4.5459    8.1674   -2.4586   -1.4621    0.     ]
 [  3.866    -2.6383    1.9242    0.10645   0.     ]
 ...
 [ -3.7503  -13.4586   17.5932   -2.7771    1.     ]
 [ -3.5637   -8.3827   12.393    -1.2823    1.     ]
 [ -2.5419   -0.65804   2.6842    1.1952    1.     ]]


The data has 1372 seperate datapoints. Each datapoint has 4 features. 

# Train-Test-Val split function

I define a function which splits a dataset into two parts based on a given proportion

In [4]:
def data_split(data, split_proportion):
    
    #Given some data, split it into a dataset and a subdataset. Both datasets have their rows shuffled.

    random.seed(0)
    np.random.seed(0)

    subdata_size    = round(split_proportion*len(data)) # Finds amount of data to put in the subdata-set
    indices         = list(np.arange(0,len(data),1))    # Makes a list from 1 to the number of samples(rows) in data
    subdata_indices = random.sample(population=indices,k=subdata_size) # Extracts a random sample of indices from this list
    
    subdata         = data[subdata_indices] # Subdata is the dataset with given indices
    dataset         = np.delete(data,subdata_indices, axis=0) # Dataset is the data but with rows removed
    np.random.shuffle(dataset) # Shuffle the dataset

    return dataset, subdata

I use the above 'data_split' function to split 'data' into 'training_data', 'test_data' and 'val_data'. The proportional sizes are 0.8, 0.16 and 0.04 respectively. 

In [5]:
training_data, val_test_data = data_split(data, split_proportion = 0.2)
val_data, test_data = data_split(val_test_data, split_proportion = 0.2)

print(training_data.shape)
print(val_data.shape)
print(test_data.shape)

(1098, 5)
(219, 5)
(55, 5)


Here i've printed the shapes of the three datasets. As we can see, their sizes are as expected from their proportional size compared to the full dataset 'data'

# Training, validation and test features / labels

The project wants our data to be separated into a data matrix X and a label vector y. Major parts of my code was written around the fact that i treat the data matrix X and the label vector y as one big matrix. So in many places in my code, functions require the full data matrix (both X and y) however within the learn() function, i will require X and y seperately as inputs then concatenate them into one full data matrix.

In [6]:
training_labels = training_data[:,-1]
val_labels = val_data[:,-1]
test_labels = test_data[:,-1]

training_features = training_data[:,:-1]
val_features = val_data[:,:-1]
test_features = test_data[:,:-1]

Now we have separated the data into two parts, where X is subscripted by the name 'features' and the labels are subscripted with the name 'labels'.

# 1.1 - 1.2 Implement a decision tree learning algorithm from scratch + Add gini index

# Label uniformity

Here i create a function which takes some data as input, and checks if the labels in the data are uniform (if there is only one unique label). If thats the case, return True, otherwise False. 

In [7]:
def check_label_uniformity(data): 
    
    # Finds all the unique labels. We know the labels occupy the last column of the data
    unique_labels = np.unique(data[:,-1]) 
    
    # If there is only one unique label, the list will have one element, then return true, otherwise false
    if len(unique_labels) == 1: 
        return True
    else:
        return False

# Feature uniformity

Here i create a function which takes some data as input, and checks wether or not all the feature values for each column are identical, if so return true, otherwise false.

In [8]:
def check_feature_uniformity(data):
    
    rows, columns = data.shape
    
    # Initialize the number of uniform columns
    num_uniform_feature_cols = 0
    
    # We skip over the last column as it is the label column
    for column in range(columns-1):
        
        # What are the unique values for the given column
        unique_features = np.unique(data[:,column])
        
        # If there is only one unique value, then the column is uniform
        if len(unique_features) == 1:
            num_uniform_feature_cols += 1
    
    # If the number of uniform columns are equal to the amount of feature columns, then the dataset
    # has a uniform featureset
    if num_uniform_feature_cols == (columns-1):
        return True
    
    else: 
        return False

# Classification

Here i create a function which takes some data as input and outputs a label prediction. This function will base its classification off the dominant label of the data. Obviously if the data has only one unique label, it will return a certain classification. However, if there are several different labels in the data, it will return the predicted label as the most dominant label in the given data.

In [9]:
def classify_data(data):
    
    # We know that the labels occupy the last column of the data
    label_column = data[:,-1]
    
    # Finds all the unique labels and the amount for each
    unique_labels, label_counts = np.unique(label_column, return_counts = True) 
    
    # Finds the label with the highest count, then finds the index of the most majority label in unique_labels.
    # Thus the majority label is the one with the highest count.
    majority_label_count = np.amax(label_counts) 
    majority_label_index = np.where(label_counts == majority_label_count) 
    majority_label = unique_labels[majority_label_index] 
    
    classification = majority_label
    
    # HOWEVER, i ran into a problem in the code where classify_data() would occationally return 2 values.
    # This is because sometimes the data is such that theres an equal amount of each label.
    # Thus when such a conflict occurs, select one label at random. Also i need that the classification variable is
    # a 1x1 numpy array.
    
    np.random.seed(0)
    if len(classification) > 1:
        classification = np.array([np.random.choice(classification)])
    
    return classification

# Find all the ways to split the feature data

I now create a function which finds all the possible ways to partition a given dataset. I will call it partitioning as opposed to splitting to avoid confusion, as the word split has already been used by the 'data_split' function.

This function takes some data, and for each column, checks for all unique values and sorts them from smallest to largest (by using the np.unique() function), then for each such unique value, it computes the midpoint between each two consecutive values and enters them into an array called splits.

Splits is then stored into a dictionary called 'possible_partitions' with the key of the numerical value of the current column. After it has iterated over all feature colums (not the label column obviously), it returns the dictionary.

I use a dictionary here as opposed to an array because given a dataset with N rows, different columns might not necessarily have exactly N-1 ways to partition the data into two parts. Some colums might have many repeat values.

In [10]:
def find_possible_partitions(data):
    
    # I want to use a dictionary here because the data can have different amounts of splits per column
    possible_partitions = {} 
    
    # I get the number of colums for the dataset by checking entry 2 in the .shape vector 
    columns = data.shape[1] 
    
    # We dont want to split by the label column so we skip the last column
    for column in range(columns - 1):
        
        # There might be repeat values in the column, so i will only look at unique values. This will also sort the column
        unique_rows = np.unique(data[:,column]) 
        splits = []
        
        # We skip the last unique row because we will partition by the middle of two points (N-1 splits for N rows)
        for row in range(len(unique_rows) - 1): 

            value = unique_rows[row]
            next_value = unique_rows[row + 1]
            inbetween_value = (value + next_value) / 2 # Select partition inbetween two consecutive unique values

            # Append the median to the splits list
            splits.append(inbetween_value)

        # Appends the list of splits to the dictionary with the name(key) of the column number
        possible_partitions[column] = splits 


    return possible_partitions

# Partition data based on column and specified value

Here i create a function which takes some dataset, a 'branch_column' value and a 'branch_value' as input and outputs two different datasets, a lower branch and an upper branch. The upper branch includes all rows from the dataset, where the value in column number 'branch_column' lays above the value 'branch_value'. The opposite holds for the lower branch. This function in other words splits the data into two parts based on what column we want to split on and what value you want to split by.

In [11]:
def data_branches(data, branch_column, branch_value):
    
    column = data[:,branch_column] # Data in the given column
    rows, columns = data.shape # Number of rows and columns

    lower_branch_data = np.zeros([1,columns]) # Need to initialize empty arrays for the code to work
    upper_branch_data = np.zeros([1,columns]) # These are zero vectors with one row and 'columns' number of columns

    # If the value in the given row and column is less or equal to the branch_value, then we put it into the lower branch,
    # else if the value is greater, we put it into the upper branch.
    # We cycle through all the rows to partition all the datapoints.
    for row in range(rows):

        if (data[row,branch_column] <= branch_value):
            lower_branch_data = np.concatenate((lower_branch_data, [data[row]]), axis = 0)
        elif (data[row,branch_column] > branch_value):
            upper_branch_data = np.concatenate((upper_branch_data, [data[row]]), axis = 0)

    upper_branch_data = np.delete(upper_branch_data, 0, axis = 0)# Delete the first row as we initialized the upper 
    lower_branch_data = np.delete(lower_branch_data, 0, axis = 0)# and lower branches with an empty row

    return lower_branch_data, upper_branch_data

# Compute the Entropy

Here i define some functions to compute the entropy of given data. First i get a list of all the labels in the data. Then i find all the unique labels and the amount of each one. Then i loop over all unique labels and i use the definition for computing the entropy, to compute the entropy of the given data.

In [12]:
def compute_entropy(data):
    
    label_data = data[:,-1] # Collects a list of all the labels
    unique_labels, label_counts = np.unique(label_data, return_counts = True) # Gives all the unique labels and their count

    entropy = 0

    for i in range(len(unique_labels)):
        
        # Probability to pick the label unique_label[i] from the list label_data
        probability = label_counts[i] / len(label_data) 
        
        # Use the definition of entropy to compute the entropy of data
        entropy -= probability*np.log2(probability) 
        
    return entropy

Here i define a function which can compute the entropy of a given partition. It takes a lower and upper branch as inputs then outputs the branching entropy, i.e the entropy of the given partitioning. 

In [13]:
def compute_branch_entropy(upper_branch, lower_branch):
    
    # We want to compute the entropy of the given branching

    p_upper_branch = len(upper_branch) / (len(upper_branch) + len(lower_branch)) # Probability of upper branch
    p_lower_branch = len(lower_branch) / (len(upper_branch) + len(lower_branch)) # Probability of lower branch

    H_upper_branch = compute_entropy(upper_branch) # Computes entropy of dataset
    H_lower_branch = compute_entropy(lower_branch)
    
    # Use the definition of conditional entropy
    branch_entropy = p_upper_branch*H_upper_branch + p_lower_branch*H_lower_branch 

    return branch_entropy

# Compute the Gini index

Here i define some functions to compute the gini index of given data. First i get a list of all the labels in the data. Then i find all the unique labels and the amount of each one. Then i loop over all unique labels and i use the definition for the gini index, to compute the gini index of the given data.

In [14]:
def compute_gini_index(data):
    
    label_data = data[:,-1] # Collects a list of all the labels
    unique_labels, label_counts = np.unique(label_data, return_counts = True) # Gives all the unique labels and their count

    gini_idx = 0

    for i in range(len(unique_labels)):
        
        # Probability to pick the label unique_label[i] from the list label_data
        probability = label_counts[i] / len(label_data) 
        
        # Use the definition of the gini index
        gini_idx += probability*(1-probability)
    
    return gini_idx

Here i define a function which can compute the gini index of a given partition. It takes a lower and upper branch as inputs then outputs the branching gini, i.e the gini index of the given partitioning. 

In [15]:
def compute_branch_gini_index(lower_branch, upper_branch):
    
    # We want to compute the gini index of the given branching

    p_upper_branch = len(upper_branch) / (len(upper_branch) + len(lower_branch)) # Probability of upper branch
    p_lower_branch = len(lower_branch) / (len(upper_branch) + len(lower_branch)) # Probability of lower branch

    G_upper_branch = compute_gini_index(upper_branch) # Computes gini index of dataset
    G_lower_branch = compute_gini_index(lower_branch)
    
    # Compute the weighted sum of the gini indices
    branch_gini_idx = p_upper_branch*G_upper_branch + p_lower_branch*G_lower_branch 

    return branch_gini_idx

# Determine optimal branching for entropy

The following function takes some data and the 'potential_splits' dictionary as inputs, and outputs the most optimal column and the most optimal value to partition the data. The function also outputs the lowest entropy, however this will only be used for an example below.

The most optimal partitioning is the one in which the labels in the upper and lower branches have the lowest overall entropy on average. Here i do not bother to compute the maximal amount of information gain, because it gives the same answer as finding the lowest overall entropy of partitioning.

In [16]:
def determine_optimal_branching(data, potential_splits):
    
    rows, columns = data.shape

    i = 0

    for column in range(columns - 1): # Skip the last column as we will not partition on the label column
        split_values = potential_splits[column] # Branch values to check
        for split in range(len(split_values)):
            
            # Retrieves a lower branch and an upper branch by calling the data_branches() function with and 
            # partitioning on a given column and split_value
            lower_branch, upper_branch = data_branches(data, branch_column = column, branch_value = split_values[split])
            
            # Computes the entropy of the partitioning 
            entropy_of_branching = compute_branch_entropy(upper_branch, lower_branch)

            if i == 0: # Condition to ensure we have a startingpoint of comparison
                lowest_entropy = entropy_of_branching

                optimal_branch_column = column
                optimal_branch_value = split_values[split]          

            # If the branch partitioning offers a lower entropy, record the new lowest entropy
            # Also record in what column and with what split_value this was achieved
            elif entropy_of_branching < lowest_entropy: 
                lowest_entropy = entropy_of_branching   

                optimal_branch_column = column
                optimal_branch_value = split_values[split]

            # Increment i by 1 so that we skip the startingpoint condition
            i += 1

    return optimal_branch_column, optimal_branch_value, lowest_entropy

### Example and test

Just to test, i will print the lowest entropy, the optimal branch column and the optimal branch value. This is done by calling on the 'find_possible_partitions()' function on our training data, then i call on the 'determine_optimal_branching()' function. Recall, in this case training_data means a matrix consisting of both X features and y labels.

In [17]:
potential_splits = find_possible_partitions(training_data)
optimal_branch_column, optimal_branch_value, lowest_entropy = determine_optimal_branching(training_data, potential_splits)

In [18]:
print('The lowest entropy is: ' + str(lowest_entropy))
print('The optimal branch column: ' + str(optimal_branch_column))
print('The optimal value to branch by is: ' + str(optimal_branch_value))

The lowest entropy is: 0.5633805580049813
The optimal branch column: 0
The optimal value to branch by is: 0.295535


As we see, the best way to partition the data (via lowest entropy) is by splitting it in two by column number 0 and by value 0.295535. Thus every row in the upper_branch will all have the values in column 0 be above 0.295535, and every row in the lower_branch will have the values in column 0 be below 0.295535. The entropy of this partitioning is 0.5634.

# Determine optimal branching for the gini index

Instead of modifying the previous 'determine_optimal_branching()' function to accomodate the gini index method, i will make a different function instead.

In [19]:
def determine_optimal_branching_gini(data, potential_splits):
    
    rows, columns = data.shape

    i = 0

    for column in range(columns - 1): # Skip the last column as we will not partition on the label column
        split_values = potential_splits[column] # Branch values to check
        for split in range(len(split_values)):
            
            # Retrieves a lower branch and an upper branch by calling the data_branches() function with and 
            # partitioning on a given column and split_value
            lower_branch, upper_branch = data_branches(data, branch_column = column, branch_value = split_values[split])
            
            # Computes the entropy of the partitioning 
            gini_of_branching = compute_branch_gini_index(upper_branch, lower_branch)

            if i == 0: # Condition to ensure we have a startingpoint of comparison
                lowest_gini = gini_of_branching

                optimal_branch_column = column
                optimal_branch_value = split_values[split]          

            # If the branch partitioning offers a lower gini index, record the new lowest gini
            # Also record in what column and with what split_value this was achieved
            elif gini_of_branching < lowest_gini: 
                lowest_gini = gini_of_branching   

                optimal_branch_column = column
                optimal_branch_value = split_values[split]

            # Increment i by 1 so that we skip the startingpoint condition
            i += 1

    return optimal_branch_column, optimal_branch_value

# Decision tree nodes

I create a leaf class. All the leaf class does, is to hold the property .predictions which gives a label prediction.

In [20]:
class Leaf:

    def __init__(self, label):
        
        self.predictions = label

The Decision_node class holds the properties .question, .lower_branch_data, .upper_branch_data, lower_branch_node and upper_branch_node. The question variable refers to what question the decision node should "ask" the new data. The upper and lower data variables contains the all the rows for the lower and upper branches, wheras the upper and lower nodes contain the upper and lower subtrees. 

In [21]:
class Decision_node:

    def __init__(self, question, lower_branch_data, upper_branch_data, lower_branch_node, upper_branch_node, majority_label):
        
        self.question = question
        self.lower_branch_node = lower_branch_node
        self.upper_branch_node = upper_branch_node
        self.lower_branch_data = lower_branch_data
        self.upper_branch_data = upper_branch_data
        self.majority_label = majority_label

# Building the tree

Now finally, we will recursively build the decision tree. The build_tree() function takes training features and training labels as input and returns either a Leaf object or a Decision_node object. If all the labels in data are the same (only one unique label), it will return a leaf with that label. If all the features in each feature column are identical, it will return a leaf with the majority label.

Otherwise, it will call upon the 'find_possible_splits()' function to find all the possible ways to partition the data. Depending on the impurity measure being 'entropy' or 'gini', the function will call upon either the determine_optimal_branching() function or determine_optimal_branching_gini() function to find the optimal column to branch by and the optimal value to branch by. Then it defines the variable 'question' as a list which contains the optimal branch column, a comparison operator and the optimal branch value. This will be used later for printing the tree and also for classifying new data.

Then the function gets the data which lands in the upper and lower branches with the 'data_branches()' function. Finally the function recursively calls upon itself by calling on the 'build_tree' function with the lower- and upper branches to create the subtrees 'upper_branch_node' and 'lower_branch_node'.

In [22]:
def build_tree(X_training, y_training, impurity_measure='entropy'):
    
    data = np.column_stack((X_training, y_training))
    majority_label = classify_data(data)
    
    # If all labels in given data is the same, return leaf. Store the majority label in this leaf.
    # In this case, the labels are all the same value, so the majority label is certain
    if check_label_uniformity(data) == True: 
        return Leaf(majority_label)
    
    # If all features in given data is the same, return leaf. Store the majority label in this leaf.
    # In this case, the labels can have different values, and thus the majority label can have some uncertainty
    if check_feature_uniformity(data) == True:
        return Leaf(majority_label)
    
    # Finds all possible splits for each column in the given data. Stores it in a dictionary.
    potential_splits = find_possible_partitions(data)
    
    
    # If impurity_measure is given as 'entropy', then we find the optimal branch column and branch value by
    # minimizing the entropy.
    # If impurity_measure is given as 'gini', then we find the optimal branch column and branch value by
    # computing the gini index
    if impurity_measure == 'entropy':
        optimal_branch_column, optimal_branch_value, lowest_entropy = determine_optimal_branching(data, potential_splits)
    
    elif impurity_measure == 'gini':
        optimal_branch_column, optimal_branch_value = determine_optimal_branching_gini(data, potential_splits)
    
    
    # Question stores the column we split on and what value to split by. Will be used for printing the tree and prediction
    question = [optimal_branch_column, '<=', optimal_branch_value] 
    
    # Values which lie below the given 'optimal_branch_value', goes to the lower branch,
    # the values which lie above go to the upper branch
    lower_branch_data, upper_branch_data = data_branches(data, optimal_branch_column, optimal_branch_value)
    
    
    # As our learn() function wants a data matrix X and a label vector y, we need to split the data matrices
    lower_branch_features = lower_branch_data[:,:-1]
    lower_branch_labels = lower_branch_data[:,-1]
    upper_branch_features = upper_branch_data[:,:-1]
    upper_branch_labels = upper_branch_data[:,-1]
    
    

    # Recursively build the lower branch.
    lower_branch_node = build_tree(lower_branch_features, lower_branch_labels, impurity_measure)

    # Recursively build the upper branch.
    upper_branch_node = build_tree(upper_branch_features, upper_branch_labels, impurity_measure)

    # Return a decision node. Store the question, the upper branch and lower branch in this node.
    return Decision_node(question, lower_branch_data, upper_branch_data,
                         lower_branch_node, upper_branch_node, majority_label)

# The first learn() function

I now create the learn function we wanted. This code might seem a bit redundant as it does not do anything but calling the build_tree() function, however it will become apparent why i create it like this once we get to implementing a pruning algorithm.

In [23]:
def learn(X, y, impurity_measure='entropy'):

    X_training = X
    y_training = y

    return build_tree(X_training, y_training, impurity_measure)

I want a way to print the decision tree in such a way i can check if the results are reasonable. In the following function we take a node (the decision tree, a sub tree or a single node) as input and then prints the information contain within the node. If the node is a Leaf node, we will print a label prediction. If the node is a decision node, we print the question contained within the decision node, then recursively call upon the 'tree_printer()' function for the lower and upper branches contained within the current node. 

In [24]:
def tree_printer(node, indent=""):
    
    # The indent variable is here so that each time the tree funciton is called, the indent
    # grows by another space. That has the effect of adding an extra indentation 
    # to each layer of the tree, making it easier to read and interpret.
    
    # If the node we are looking at is a leaf node, print the prediction for the label.
    if isinstance(node, Leaf):
        print (indent + "Predict", node.predictions)
        return

    # Otherwise print the question at the decision node
    print(indent + 'Is value at column: ' + str(node.question[0]) + str(node.question[1]) + str(node.question[2]) + '?')

    # Recursively decend the tree down the lower branch 
    print(indent + '___ Value lies below:')
    tree_printer(node.lower_branch_node, indent + "  ")

    # Recursively decend the tree down the upper branch
    print(indent + '^^^ Value lies above:')
    tree_printer(node.upper_branch_node, indent + "  ")

### Example of training and visualizing a tree

Here i show how to train a decision tree and how to print it

In [25]:
my_tree = learn(training_features, training_labels)

In [26]:
tree_printer(my_tree)

Is value at column: 0<=0.295535?
___ Value lies below:
  Is value at column: 1<=5.86535?
  ___ Value lies below:
    Is value at column: 2<=6.21865?
    ___ Value lies below:
      Is value at column: 1<=4.09405?
      ___ Value lies below:
        Predict [1.]
      ^^^ Value lies above:
        Is value at column: 1<=4.1572?
        ___ Value lies below:
          Predict [0.]
        ^^^ Value lies above:
          Predict [1.]
    ^^^ Value lies above:
      Is value at column: 1<=-3.16705?
      ___ Value lies below:
        Is value at column: 0<=-0.357315?
        ___ Value lies below:
          Predict [1.]
        ^^^ Value lies above:
          Predict [0.]
      ^^^ Value lies above:
        Predict [0.]
  ^^^ Value lies above:
    Is value at column: 0<=-3.4448999999999996?
    ___ Value lies below:
      Predict [1.]
    ^^^ Value lies above:
      Predict [0.]
^^^ Value lies above:
  Is value at column: 0<=1.7438500000000001?
  ___ Value lies below:
    Is value at column

# Decision tree predictions

Now that the tree seems to work, i define a function which will predict a label. This function will read a single row from a dataset and with the given tree, predict the label.

If 'print_progress' is set to True, then it will print the relevant questions and answers during the process as the algorithm is predicting the label. 

In [27]:
def predict(data_row, node, print_progress=False):
    
    # If the remaining node is a leaf node, return the prediction of this leaf node
    if isinstance(node, Leaf):
        if print_progress == True:
            print('We have reached a leaf node, we predict the label is: ')
        return node.predictions
    
    # Printing relevant question for current node
    if print_progress == True:
        print('Is the value in column ' + str(node.question[0]) + ' ' + str(node.question[1]) 
              + ' ' + str(node.question[2]) + '?')
    
    # If the split value in the question is greater or equal to the value in the data_row at the given column
    # then go to the lower branch, otherwise go to the upper branch
    if node.question[2] >= data_row[node.question[0]]:
        if print_progress == True:
            print('True')
        return predict(data_row, node.lower_branch_node, print_progress)
    else:
        if print_progress == True:
            print('False')
        return predict(data_row, node.upper_branch_node, print_progress)

### Example of how to use the predict function

Let us test this classifier function on the first datapoint in the validation dataset. I will set the variable print_progress to True, because i want to see how the classifier got to its conclusion.

In [28]:
example = val_data[0]

print('Let us predict the label of the following example; \n')
print(str('[Column 0, Column 1, Column 2, Column 3, Label]'))
print(str(example) + '\n')
print(predict(example, my_tree, print_progress=True))

if predict(example, my_tree) == example[-1]:
    print('This prediction was correct!')
else:
    print('This prediction was false!')

Let us predict the label of the following example; 

[Column 0, Column 1, Column 2, Column 3, Label]
[-3.8053  2.4273  0.6809 -1.0871  1.    ]

Is the value in column 0 <= 0.295535?
True
Is the value in column 1 <= 5.86535?
True
Is the value in column 2 <= 6.21865?
True
Is the value in column 1 <= 4.09405?
True
We have reached a leaf node, we predict the label is: 
[1.]
This prediction was correct!


# Decision tree accuracy

We will now try to classify all the datapoints in the validation dataset and then check if the decision tree 
predicted the correct label or not. This function will just loop over each row in a dataset, and then use the above predict() function to count how many incorrect and correct predictions there were then return an accuracy.

In [29]:
def predict_data_accuracy(data_features, data_labels, node):
    
    # Get the amount of rows in the feature data
    rows = data_features.shape[0]
    

    # Initialize amount of correct and incorrect predictions
    correct_predictions = 0
    incorrect_predictions = 0
    
    incorrect_rows = []

    for row in range(rows):

        # The classify function returns an np.array with a single element so we just extract the value
        prediction = predict(data_features[row], node)[0]

        # If the prediction is equal to the actual label, then we get a correct prediction
        if prediction == data_labels[row]:
            correct_predictions += 1

        # If not, we have an incorrect prediction
        else:
            incorrect_predictions += 1
            incorrect_rows.append(row)

    # The accuracy of our decision tree is then 
    accuracy = correct_predictions / (correct_predictions + incorrect_predictions)
    
    print('Correct predictions: ' + str(correct_predictions))
    print('Incorrect predictions: ' + str(incorrect_predictions))
    print('The incorrect predictions happened for datapoints: ' + str(incorrect_rows))
    
    return accuracy

### Accuracy of decision tree with the entropy method

In [30]:
predict_data_accuracy(val_features,val_labels, my_tree)

Correct predictions: 215
Incorrect predictions: 4
The incorrect predictions happened for datapoints: [14, 46, 124, 214]


0.9817351598173516

### Accuracy of decision tree with the gini index method

I want to test the decision tree algorithm using the gini index instead of the entropy. 

In [31]:
my_2nd_tree = learn(training_features, training_labels, impurity_measure='gini')

In [32]:
predict_data_accuracy(val_features,val_labels, my_2nd_tree)

Correct predictions: 214
Incorrect predictions: 5
The incorrect predictions happened for datapoints: [46, 82, 124, 129, 214]


0.9771689497716894

As we can see, in this specific case, the Gini index method of determining optimal partitioning gives us a lower degree of accuracy.

# 1.3 Add reduced-error pruning

# Reduced error pruning

I first define a function which can compute wether or not replacing a given subtree by a leaf with the majority label is benefitial for the pruning accuracy or not.

In [33]:
def prune_subtree_benefit(prune_features, prune_labels, question, majority_label, lower_branch_node, upper_branch_node):
    
    # I want the pruning data to be one large matrix with the first columns being feature columns and the last being
    # a label column
    prune_data = np.column_stack((prune_features,prune_labels))
    
    # The prediction of the current node is the majority label here
    prediction_leaf = majority_label
    
    # Initialize amount of correct and incorrect predictions for pruned subtree
    correct_pruned = 0
    incorrect_pruned = 0
    
    # Initialize amount of correct and incorrect predictions for unpruned subtree
    correct = 0
    incorrect = 0
    
    # For each row in the pruning data, check if the value of the prune data is above or below the branch_value
    # contained in question[2], then make the prediction on either the upper or lower branch. If the prediction
    # is correct, increment correct by 1, if not increment incorrect by 1
    # if the prediction by pruning the subtree (prediction_leaf) is correct, increment correct_pruned by 1
    # if not increment incorrect_pruned by 1. 
    for row in range(len(prune_labels)):

        if question[2] >= prune_data[row,question[0]]:
            prediction = lower_branch_node.predictions
        else:
            prediction = upper_branch_node.predictions   
        
        # Increment amount of correct / incorrect predictions for original subtree
        if prediction == prune_labels[row]:
            correct += 1
        else:
            incorrect += 1
        
        # Increment amount of correct / incorrect predictions for pruned subtree
        if prediction_leaf == prune_labels[row]:
            correct_pruned += 1
        else:
            incorrect_pruned += 1
    
    # Compute accuracy
    prune_accuracy_pruned = correct_pruned / (correct_pruned + incorrect_pruned)
    prune_accuracy = correct / (correct + incorrect)

    # If the accuracy of the pruned tree is the same or better, return True, if not, False
    if prune_accuracy_pruned >= prune_accuracy:
        return True
    else:
        return False
    

I now want to modify the 'build_tree()' function. It should build a tree using training data, then use pruning data to replace each subtree in the tree if the accuracy of the pruning data is not adversly affected by replacing the subtree by a leaf node containing the majority label of the subtree.

In [34]:
def build_tree_prune(X_training,y_training, X_prune, y_prune, impurity_measure='entropy', prune=False):
    
    data = np.column_stack((X_training, y_training))
    prune_data = np.column_stack((X_prune, y_prune))
    
    majority_label = classify_data(data)
    
    # If all labels in given data is the same, return leaf. Store the data in this leaf.
    if check_label_uniformity(data) == True: 
        return Leaf(majority_label)
    
    # If all features in given data is the same, return leaf. Store the majority label in this leaf.
    # In this case, the labels can have different values, and thus the majority label can have some uncertainty
    if check_feature_uniformity(data) == True:
        return Leaf(majority_label)
    
    # Finds all possible splits for each column in the given data. Stores it in a dictionary.
    potential_splits = find_possible_partitions(data)
    
    
    # If impurity_measure is given as 'entropy', then we find the optimal branch column and branch value by
    # minimizing the entropy.
    # If impurity_measure is given as 'gini', then we find the optimal branch column and branch value by
    # computing the gini index
    if impurity_measure == 'entropy':
        optimal_branch_column, optimal_branch_value, lowest_entropy = determine_optimal_branching(data, potential_splits)
    
    elif impurity_measure == 'gini':
        optimal_branch_column, optimal_branch_value = determine_optimal_branching_gini(data, potential_splits)
    
    
    
    # Question stores the column we split on and what value to split by. Will be used for printing the tree and prediction
    question = [optimal_branch_column, '<=', optimal_branch_value] 
    
    # Values which lie below the given 'optimal_branch_pruneue', goes to the lower branch,
    # the values which lie above go to the upper branch
    lower_branch_data, upper_branch_data = data_branches(data, optimal_branch_column, optimal_branch_value)
    lower_branch_prune, upper_branch_prune = data_branches(prune_data, optimal_branch_column, optimal_branch_value)
    
    
    # As our learn() function wants a data matrix X and a label vector y, we need to split the data matrices
    lower_branch_features = lower_branch_data[:,:-1]
    lower_branch_labels = lower_branch_data[:,-1]
    upper_branch_features = upper_branch_data[:,:-1]
    upper_branch_labels = upper_branch_data[:,-1]
    
    
    lower_branch_prune_features = lower_branch_prune[:,:-1]
    lower_branch_prune_labels = lower_branch_prune[:,-1]
    upper_branch_prune_features = upper_branch_prune[:,:-1]
    upper_branch_prune_labels = upper_branch_prune[:,-1]
    

    # Recursively build the lower branch.
    lower_branch_node = build_tree_prune(lower_branch_features, lower_branch_labels,
                                      lower_branch_prune_features, lower_branch_prune_labels, impurity_measure, prune)

    # Recursively build the upper branch.
    upper_branch_node = build_tree_prune(upper_branch_features, upper_branch_labels,
                                      upper_branch_prune_features, upper_branch_prune_labels, impurity_measure, prune)
    
    # If both the lower and upper nodes are leaf nodes, and if we have any pruning data that has reached this point,
    # then compute the pruning benefit. If pruning has a benefit on the accuracy of the pruning data, then
    # make the current node into a leaf node with the majority label of the training data at this point.
    if (prune == True) and isinstance(lower_branch_node, Leaf) and isinstance(upper_branch_node, Leaf) and (len(prune_data[:,-1]) > 0):
        pruning_benefit = prune_subtree_benefit(prune_data[:,:-1], prune_data[:,-1], question,
                                                majority_label, lower_branch_node, upper_branch_node)
        if pruning_benefit == True:
            return Leaf(majority_label)

    
    # Return a decision node. Store the question, the upper branch and lower branch in this node.
    return Decision_node(question, lower_branch_data, upper_branch_data,
                         lower_branch_node, upper_branch_node, majority_label)

# The learn() function

The project asks us to have a learn function with the inputs X, y, impurity_measure and prune. Since i only want to split the training data into pruning data once, then i need to have the tree_builder_prune() function seperate from the learn() functon. This function simply stacks X and y back together into a big matrix, splits them into a training set and a pruning set, then we remove the features from the labels for both sets. Then we call upon the tree_builder_function().

In [35]:
def learn(X, y, impurity_measure='entropy', prune=False):
    
    # The ratio of prune data by amount of total training data
    split_proportion = 0.25
    
    # I want to combine the features and labels before splitting
    data = np.column_stack((X, y))
    
    # Split the data into training data and purning data
    training_data, prune_data = data_split(data, split_proportion)
    
    # Split the matrices back into feature matrices and label vectors
    X_training = training_data[:,:-1]
    y_training = training_data[:,-1]
    
    X_prune = prune_data[:,:-1]
    y_prune = prune_data[:,-1]
    
    # Build the tree
    return build_tree_prune(X_training, y_training, X_prune, y_prune, impurity_measure, prune)

# 1.4 Evaluate your algorithm

We now want to evaluate the decision tree using validation data and vary the hyperparameters and then select the most suitable model. We will train the decision tree using training data X_training and y_training. Recall that inside the learn() function, 25% of this training data is removed from the training data and instead used as pruning data. Then, once the tree is built, we will call the function predict_data_accuracy() with \teval\_features} \textbf{val\_labels} and the decision tree, to compute the trees accuracy on the validation data. I will vary the hyperparameters \textbf{impurity\_measure} and \textbf{prune} and check the accuracy for all the four combinations

# Pruned vs Unpruned & Entropy vs Gini Trees

In [36]:
my_tree = learn(training_features, training_labels, impurity_measure='entropy', prune=False)

In [37]:
my_tree_pruned = learn(training_features, training_labels, impurity_measure='entropy', prune=True)

In [38]:
print('Correct and incorrect predictions for pruned tree: ')
accuracy_pruned = predict_data_accuracy(val_features, val_labels, my_tree_pruned)
print('\n Correct and incorrect predictions for non-pruned tree: ')
accuracy_nonpruned = predict_data_accuracy(val_features, val_labels, my_tree)

difference = accuracy_pruned - accuracy_nonpruned

print('\n Accuracy pruned tree: ')
print(accuracy_pruned)
print('\n Accuracy non-pruned tree: ')
print(accuracy_nonpruned)
print('\n Accuracy difference: ')
print(difference)

Correct and incorrect predictions for pruned tree: 
Correct predictions: 215
Incorrect predictions: 4
The incorrect predictions happened for datapoints: [46, 82, 124, 214]

 Correct and incorrect predictions for non-pruned tree: 
Correct predictions: 213
Incorrect predictions: 6
The incorrect predictions happened for datapoints: [46, 82, 95, 103, 124, 214]

 Accuracy pruned tree: 
0.9817351598173516

 Accuracy non-pruned tree: 
0.9726027397260274

 Accuracy difference: 
0.0091324200913242


In [39]:
my_tree_gini = learn(training_features, training_labels, impurity_measure='gini', prune=False)

In [40]:
my_tree_pruned_gini = learn(training_features, training_labels, impurity_measure='gini', prune=True)

In [41]:
print('Correct and incorrect predictions for pruned tree: ')
accuracy_pruned = predict_data_accuracy(val_features, val_labels, my_tree_pruned_gini)
print('\n Correct and incorrect predictions for non-pruned tree: ')
accuracy_nonpruned = predict_data_accuracy(val_features, val_labels, my_tree_gini)

difference = accuracy_pruned - accuracy_nonpruned

print('\n Accuracy pruned tree: ')
print(accuracy_pruned)
print('\n Accuracy non-pruned tree: ')
print(accuracy_nonpruned)
print('\n Accuracy difference: ')
print(difference)

Correct and incorrect predictions for pruned tree: 
Correct predictions: 215
Incorrect predictions: 4
The incorrect predictions happened for datapoints: [46, 82, 124, 214]

 Correct and incorrect predictions for non-pruned tree: 
Correct predictions: 214
Incorrect predictions: 5
The incorrect predictions happened for datapoints: [46, 82, 124, 129, 214]

 Accuracy pruned tree: 
0.9817351598173516

 Accuracy non-pruned tree: 
0.9771689497716894

 Accuracy difference: 
0.004566210045662156


In [42]:
tree_printer(my_tree)

Is value at column: 0<=0.746345?
___ Value lies below:
  Is value at column: 1<=5.3646?
  ___ Value lies below:
    Is value at column: 0<=-0.36205?
    ___ Value lies below:
      Is value at column: 2<=6.21865?
      ___ Value lies below:
        Predict [1.]
      ^^^ Value lies above:
        Is value at column: 1<=-2.2431270000000003?
        ___ Value lies below:
          Predict [1.]
        ^^^ Value lies above:
          Predict [0.]
    ^^^ Value lies above:
      Is value at column: 2<=2.1670999999999996?
      ___ Value lies below:
        Is value at column: 2<=0.47011500000000006?
        ___ Value lies below:
          Predict [1.]
        ^^^ Value lies above:
          Is value at column: 1<=-0.4242105?
          ___ Value lies below:
            Predict [1.]
          ^^^ Value lies above:
            Predict [0.]
      ^^^ Value lies above:
        Predict [0.]
  ^^^ Value lies above:
    Is value at column: 0<=-3.35385?
    ___ Value lies below:
      Predict [1.]


In [43]:
tree_printer(my_tree_pruned)

Is value at column: 0<=0.746345?
___ Value lies below:
  Is value at column: 1<=5.3646?
  ___ Value lies below:
    Is value at column: 0<=-0.36205?
    ___ Value lies below:
      Is value at column: 2<=6.21865?
      ___ Value lies below:
        Predict [1.]
      ^^^ Value lies above:
        Is value at column: 1<=-2.2431270000000003?
        ___ Value lies below:
          Predict [1.]
        ^^^ Value lies above:
          Predict [0.]
    ^^^ Value lies above:
      Is value at column: 2<=2.1670999999999996?
      ___ Value lies below:
        Is value at column: 2<=0.47011500000000006?
        ___ Value lies below:
          Predict [1.]
        ^^^ Value lies above:
          Is value at column: 1<=-0.4242105?
          ___ Value lies below:
            Predict [1.]
          ^^^ Value lies above:
            Predict [0.]
      ^^^ Value lies above:
        Predict [0.]
  ^^^ Value lies above:
    Is value at column: 0<=-3.35385?
    ___ Value lies below:
      Predict [1.]


In [44]:
tree_printer(my_tree_gini)

Is value at column: 0<=0.295535?
___ Value lies below:
  Is value at column: 1<=7.4991?
  ___ Value lies below:
    Is value at column: 0<=-0.4031?
    ___ Value lies below:
      Is value at column: 2<=6.21865?
      ___ Value lies below:
        Predict [1.]
      ^^^ Value lies above:
        Is value at column: 1<=-2.2431270000000003?
        ___ Value lies below:
          Predict [1.]
        ^^^ Value lies above:
          Predict [0.]
    ^^^ Value lies above:
      Is value at column: 1<=5.4422?
      ___ Value lies below:
        Is value at column: 2<=2.62465?
        ___ Value lies below:
          Predict [1.]
        ^^^ Value lies above:
          Is value at column: 0<=-0.365255?
          ___ Value lies below:
            Predict [1.]
          ^^^ Value lies above:
            Predict [0.]
      ^^^ Value lies above:
        Predict [0.]
  ^^^ Value lies above:
    Is value at column: 0<=-3.9402?
    ___ Value lies below:
      Predict [1.]
    ^^^ Value lies above:
 

In [45]:
tree_printer(my_tree_pruned_gini)

Is value at column: 0<=0.295535?
___ Value lies below:
  Is value at column: 1<=7.4991?
  ___ Value lies below:
    Is value at column: 0<=-0.4031?
    ___ Value lies below:
      Is value at column: 2<=6.21865?
      ___ Value lies below:
        Predict [1.]
      ^^^ Value lies above:
        Is value at column: 1<=-2.2431270000000003?
        ___ Value lies below:
          Predict [1.]
        ^^^ Value lies above:
          Predict [0.]
    ^^^ Value lies above:
      Is value at column: 1<=5.4422?
      ___ Value lies below:
        Is value at column: 2<=2.62465?
        ___ Value lies below:
          Predict [1.]
        ^^^ Value lies above:
          Predict [0.]
      ^^^ Value lies above:
        Predict [0.]
  ^^^ Value lies above:
    Is value at column: 0<=-3.9402?
    ___ Value lies below:
      Predict [1.]
    ^^^ Value lies above:
      Predict [0.]
^^^ Value lies above:
  Is value at column: 2<=-4.4434000000000005?
  ___ Value lies below:
    Is value at column: 0

# Test data performance on selected model

The best model seems to be the 'my_tree_pruned' as it has the highest accuracy. It is also the least complex tree (has the fewest amount of nodes). We now select this decision tree as the best classifier, and we test its performance on the test data.

In [46]:
accuracy = predict_data_accuracy(test_features, test_labels, my_tree_pruned)
print('The accuracy of our selected model is: ' + str(accuracy))

Correct predictions: 54
Incorrect predictions: 1
The incorrect predictions happened for datapoints: [16]
The accuracy of our selected model is: 0.9818181818181818


# 1.5 Compare to an existing implementation

# Using Sklearn

Now we can compare my model to something more robust. I will use sklearn's .DecisionTreeClassifier to make 

In [47]:
# Import what we need
from sklearn.tree import DecisionTreeClassifier

In [48]:
# Define a decision tree with criterion 'entropy' and one with criterion 'gini'
tree_sklearn = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
tree_sklearn_gini = DecisionTreeClassifier(criterion = 'gini', random_state = 0)

# Fit the two decision trees with the training data
tree_sklearn.fit(training_features, training_labels)
tree_sklearn_gini.fit(training_features, training_labels)

DecisionTreeClassifier(random_state=0)

In [49]:
# Check the accuracy of the two decision trees
accuracy_sklearn = tree_sklearn.score(val_features, val_labels)
accuracy_sklearn_gini = tree_sklearn_gini.score(val_features, val_labels)

In [50]:
# Print the accuracies
print('The accuracy of the sklearn tree was: ' + str(accuracy_sklearn))
print('The accuracy of the sklearn tree (gini) was: ' + str(accuracy_sklearn_gini))

The accuracy of the sklearn tree was: 0.9817351598173516
The accuracy of the sklearn tree (gini) was: 0.9817351598173516


We want to compare these accuracies to our selected model on the validation data.

In [51]:
accuracy_pruned = predict_data_accuracy(val_features, val_labels, my_tree_pruned)
print('The accuracy for my selected model: ' + str(accuracy_pruned))

Correct predictions: 215
Incorrect predictions: 4
The incorrect predictions happened for datapoints: [46, 82, 124, 214]
The accuracy for my selected model: 0.9817351598173516


As we see, they are all equally accurate on the validation data. However, a big problem with my implementation is that it takes around 30 seconds to train and create a decision tree. The Sklearn implementation will fit its classifier within a fraction of a second. The most time consuming part of my code is the determine_optimal_branching() function. This function needs to compute the entropy or the gini index for every single possible split between each unique value for up to a thousand rows over 4 columns (atleast for the root node). The sklearn implementation probably has a much more clever and elegant way of doing this which leads to a more efficient code.

Thus, if we had to choose, we should choose the sklearn implementation. In this specific case the accuracies of the tree_sklearn and tree_sklearn_gini are the same. I will just pick the first of the two trees as my final selected model.

# Test accuracy of final model

I now check the accuracy of my final selected model on the test data. 

In [52]:
accuracy_final_model = tree_sklearn.score(test_features, test_labels)
print(accuracy_final_model)

0.9818181818181818
